# COMPLETE TESS LIGHT CURVE INFORMATION

This notebook takes the GTO objects list and tries to first simply resolve the ambiguity, taking the first file in folder. If not, try to complete the TESS light curve information.

## Modules and configuration

### Modules

In [1]:
# Modules import:
import pandas as pd
import numpy as np
import os
from time import sleep

#from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
#from astropy import units as u
#from astropy.time import Time
from astropy.io import fits

import lightkurve as lk

#%matplotlib inline
#import matplotlib.pyplot as plt

#from pylab import rcParams
#rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [2]:
# Configuration:
DATA_FILE = "../data/CARM_VIS_objects_with_PG.csv"
TESS_LC_FOLDER = "../data/CARMENES_GTO_TESS_lc/lightcurves/"


## TESS file correction

In [3]:
gto = pd.read_csv(DATA_FILE, sep=',', decimal='.')
gto.head()

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002992,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001671,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN


In [4]:
gto['has_lc']

0      False
1      False
2       True
3       True
4      False
       ...  
385     True
386    False
387    False
388    False
389     True
Name: has_lc, Length: 390, dtype: bool

In [5]:
gto['lc_file']

0                                                    NaN
1                                                    NaN
2      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2354...
3      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2350...
4                                                    NaN
                             ...                        
385    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0018...
386                                                  NaN
387                                                  NaN
388                                                  NaN
389    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0005...
Name: lc_file, Length: 390, dtype: object

In [6]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [7]:
type(gto.loc[0, 'TIC_id'])

numpy.int64

In [8]:
#for i in range (0,20):
for i in range (136, len(gto)):
    filename = gto.loc[i, 'lc_file']
    # See if it exists:
    try:
        karmn_name = gto.loc[i, 'Karmn']
        object_name = gto.loc[i, 'Name']
        tic_id = "TIC" + str(gto.loc[i, 'TIC_id'])
        target_folder = TESS_LC_FOLDER + karmn_name + "/"
        if filename == "<ambiguous>":
            # Select the first file in folder:
            files = os.listdir(target_folder)
            selected_file = files[0]
            print("<ambiguous>: selecting the first file in %s folder: %s" 
                  %(target_folder, selected_file))
            gto.loc[i, 'lc_file'] = target_folder + selected_file
            gto.loc[i, 'has_lc'] = True
        elif filename is np.nan or not os.path.exists(filename):
            sleep(30)
            print("%s filename DOES NOT exist, trying to look and download" %filename)
            lk.search.log.setLevel("DEBUG")
            #search_result = lk.search_lightcurve(object_name, mission='TESS', exptime=600)
            search_result = lk.search_lightcurve(tic_id, mission='TESS', exptime=1800)
            if len(search_result) > 0:
                # Select the item with greater exposition time, download it and remove NaN values:
                exp_times = np.array(search_result.exptime.value)
                max_idx = np.argmax(exp_times)
                lc = search_result[max_idx].download().remove_nans()
                sector = search_result[max_idx].mission[0][-2:]
                # Store it and update the info in the file:
                if not os.path.exists(target_folder):
                    os.makedirs(target_folder)
                else:
                    pass
                target_filename = target_folder + \
                    karmn_name + "_TESS_" + tic_id + "_S" + sector + ".fits"
                lc.to_fits(path=target_filename, overwrite=True, flux_column_name='flux')
                gto.loc[i, 'lc_file'] = target_filename
                gto.loc[i, 'has_lc'] = True
                print("...SUCCESS: found object and stored file %s" %target_filename)
            else:
                # No results returned
                print("** WARNING: no results found for %s star" %karmn_name)
        else:
            print("%s filename DOES exist" %filename)
    except Exception as e:
            print("*** SOME ERROR happened. Error = %s" %str(e))
        


../data/CARMENES_GTO_TESS_lc/lightcurves/J15218+209/J15218+209_TESS_TIC355793860_S24.fits filename DOES exist


Started querying MAST for observations with the exact target_name='036853511'.


nan filename DOES NOT exist, trying to look and download


No observations found. Now performing a cone search instead.
Started querying MAST for observations within 0.0001 arcsec arcsec of objectname='TIC36853511'.
MAST found 0 observations. Now querying MAST for the corresponding data products.
No data found for target "TIC36853511".


** WARNING: no results found for J15194-077 star


...

** WARNING: no results found for J00162+198E star


Started querying MAST for observations with the exact target_name='176287658'.
MAST found 2 observations. Now querying MAST for the corresponding data products.


nan filename DOES NOT exist, trying to look and download


MAST found 0 matching data products.


** WARNING: no results found for J00067-075 star
../data/CARMENES_GTO_TESS_lc/lightcurves/J00051+457/J00051+457_TESS_TIC439946126_S17.fits filename DOES exist


In [9]:
i

389

### Check final numbers:

In [10]:
gto[gto['lc_file'] == "<ambiguous>"]

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS


In [11]:
gto[gto['has_lc'] == False][['has_lc', 'lc_file']]

,has_lc,lc_file
0,False,NaN
1,False,NaN
4,False,NaN
6,False,NaN
8,False,NaN
...,...,...
373,False,NaN
384,False,NaN
386,False,NaN
387,False,NaN


## Store the result

In [12]:
gto.to_csv(DATA_FILE, sep=',', decimal='.', index=False)

**OBSERVATIONS AND CONCLUSIONS:**
- We have tried to complete the TESS light curves.
- Noticed that downloaded files were much smaller than the existing ones, so later decided to just keep the existing ones, and only try to downlñoad if no file existed previously.
- Finally, 110 objects (about one third of the total) remained with no available TESS light curve.